In [1]:
# Import necessary packages
import os
import csv 
import re
import pandas as pd 
import numpy as np 
import nltk
from nltk.corpus import stopwords 

In [2]:
# Define file path for storing raw data
file_path = "/Users/maxwellcozean/Desktop/ECON 1680/Text Analysis Project/Data"

# Join paths and import csv as dataframe
df = pd.read_csv(os.path.join(file_path,"datasciencejobs.csv"),encoding='utf-8') 

In [3]:
# Drop index columns from csv file
df = df.drop(['Unnamed: 0','index'],axis=1)

# Rename type of ownership column 
df = df.rename(columns={'Type of ownership': 'Type of Ownership'})

# Consolidate null values into a single measure

# Replace -1 (int) as null
df.replace(-1, np.nan, inplace=True)

# Replace -1 (str) as null
df.replace('-1', np.nan, inplace=True) 

# Replace Unknown (str) as null
df.replace('Unknown', np.nan, inplace=True) 

# Replace Unknown / Non-Applicable (str) as null
df.replace('Unknown / Non-Applicable', np.nan, inplace=True)

# Keep company name (text prior to newline)
df['Company Name'] = df['Company Name'].str.split('\n', expand=True)[0]

In [4]:
# Split expected annual salary variable into minimum and maximum salary estimates
df[['Minimum Salary Estimate','Maximum Salary Estimate']] = df['Salary Estimate'].str.extract(r'\$(\d+)K\s*-\s*\$(\d+)K')

# Multiply mimimum salary estimate by 1000
df['Minimum Salary Estimate'] = pd.to_numeric(df['Minimum Salary Estimate'])*1000 

# Multiply maximum salary estimate by 1000
df['Maximum Salary Estimate'] = pd.to_numeric(df['Maximum Salary Estimate'])*1000

# Drop null values of minimum and maximum salary estimates (regression output)
df = df.dropna(subset=['Minimum Salary Estimate','Maximum Salary Estimate'])

# Create single measure of expected salary by taking the average of min/max salary estimates
df['Average Salary Estimate'] = (df['Minimum Salary Estimate']+df['Maximum Salary Estimate'])/2 

In [5]:
# Define skills dictionary
skills_dictionary = [
    'Analysis', 'Analysis and design', 'Analysis and reporting', 'Analysis skills',
    'Analysis support', 'Analysis techniques', 'Analysis to support', 'Analysis tools',
    'Analytic support', 'Analytical abilities', 'Analytical methods', 'Analytical projects',
    'Analytical results', 'Analytical support', 'Analytical techniques', 'Analytical tools',
    'Analyzing information', 'Quantitative analysis', 'Quantitative and qualitative',
    'anova', 'Advanced statistics', 'Data modeling',
    'Linear and logistic', 'Linear models', 'Predictive models', 'r-square',
    'Regression', 'Statistical analyses', 'Statistical analysis', 'Statistical data analysis',
    'Statistical methods', 'Statistical modeling', 'Statistical models', 'Statistical process',
    'Statistical reports', 'Statistical techniques', 'Statistical tests', 'Statistics',
    'Summarizing data', 'Algorithm', 'Algorithm design', 'Algorithm development', 'Algorithms',
    'Algorithms and applications', 'Algorithms and formulations', 'Algorithms in solving real',
    'Algorithms to match online', 'AMPL', 'Combinatorial optimization', 'Constraint based',
    'Constraint programming', 'Cplex', 'Decision analysis', 'Decision making', 'Decision problems',
    'Decision science', 'Decision sciences', 'Decision support', 'Decision support analysis',
    'Decision support applications', 'Decision support functions', 'Decision support models',
    'Decision support research', 'Decision support research analyst', 'Decision support software',
    'Decision support systems', 'Decision support tools', 'Decision tools', 'Decision trees',
    'Forecasting', 'Integer', 'Linear programming', 'Mathematical modeling', 'Mathematical models',
    'Mathematical programming', 'Nonlinear', 'Non-linear', 'Optimization', 'Quadratic',
    'Stochastic optimization', 'MATLAB', 'Model development', 'Model formulation', 'Modeling',
    'Network modeling', 'Simulate', 'Simulation', 'Simulations',
    'Data analysis', 'Data set', 'Data collection', 'Data gathering', 'Data integrity',
    'Data mining', 'Database management', 'Datamart', 'Dataset', 'ERD', 'etl', 'Large data',
    'Relational databases', 'Software applications', 'Macros', 'Microsoft Office', 'Microsoft suite', 
    'Microsoft Word', 'MS Office', 'MS Word', 'o365', 'Power Point', 'PowerPoint', 'Spreadsheet', 
    'Spreadsheets', 'Word processing','Cognos', 'fixml', 'd3', 'pentaho', 'powerbi', 'tableau',
    'Access', 'And access', 'cassandra', 'DB2', 'dbms', 'Hbase', 'Microsoft Access',
    'mongodb', 'MS Access', 'Mysql', 'Nosql', 'Oracle', 'SQL Server', 'SQL',
    'Teradata', 'tsql', 'XML', 'xsd', 'xsl', 'r', 'SAS', 'SPSS', 'STATA',
    'C', 'html', 'Linux', 'Pearl', 'Perl', 'pig', 'Python', 'ruby', 'VBA', 'Visual Basic',
    'Azure', 'Google Analytics', 'Hadoop', 'Hive', 'Salesforce', 'SAP', 'Watson'
    'Accredited college', 'BA', 'Bachelor of business administration', 'Bachelor of science',
    'Bachelors', 'Bachelors degree', 'BS', 'College or university',
    'Advanced degree', 'Doctorate', 'Graduate degree', 'Masters', 'Masters ', 'MS degree',
    'Master of business administration', 'MBA', 'MBA degree', 'Ph', 'PhD', 'PhD degree']

In [6]:
# Drop null values of job description
df = df.dropna(subset=['Job Description'])

# Define a translation table for the punctuation to be removed
table_punctuation = str.maketrans('', '', '!"#$%&\'’*+()“”–,-./:;<=>?@[\\]^_`{|}~•')

# Define set of english stopwords to be removed 
nltk_stopwords = set(stopwords.words('english'))

# Define list of stopwords to include
include = ['and', 'to', 'or']

# Define list of stopwords to remove (all but include)
stops = [word for word in nltk_stopwords if word not in include]

# Define list of link keywords 
links = ['https', 'www']

# Initialize empty list of cleaned job descriptions
clean_description = []

# Initialize empty list of extracted skills
skills_list = []

# For each job description in the dataframe 
for i, row in enumerate(df['Job Description']):
    
    # Remove punctuation using the translation table defined above
    description = row.translate(table_punctuation)
    
    # Remove carriage returns from the current description
    description = description.replace('\r',' ')
    
    # Remove newlines from the current description
    description = description.replace('\n',' ')
    
    # Remove double spaces (replace with single space)
    description = description.replace('  ',' ')
    
    # Remove stopwords 
    tokens = [word.lower() for word in nltk.tokenize.word_tokenize(description) if word.lower() not in stops]
    
    # Remove tokens with links
    tokens = [word for word in tokens if not any(link in word for link in links)]
    
    # Append tokens (with a space) to the cleaned description for each posting
    clean_description.append(' '.join(tokens))
    
    # Intitialize an empty dictionary for the counts of skills within each job description
    skill_counts = {}
    
    # For each skill in the skills dictionary
    for skill in skills_dictionary:
        
        # Update dictionary with the skills found in both the current description 
        # and the skills dictionary and the number of times they appear
        skill_counts[skill] = len(re.findall(r'\b{}\b'.format(re.escape(skill)), description, flags=re.IGNORECASE))
    
    # Initialize empty list of skills for current posting
    job_skills = []
    
    # For each skill and the number of times that skill appears in the job description
    for skill, count in skill_counts.items():
        
        # Add skills that appear multiple times by extending the job skills list
        job_skills.extend([skill]*count)
    
    # Append skills from current description to list of skills from all descriptions
    skills_list.append(job_skills)
    
    # Track progress
    # print(round((i / len(df) * 100),3),'%')
    
# Create column for the cleaned job description
df['Cleaned Job Description'] = clean_description

# Create column of extracted skills for each posting
df['Skills'] = skills_list

In [7]:
# Drop columns irrelevant to analysis 
df = df.drop(['Company Name','Rating','Salary Estimate','Location',
              'Headquarters','Size','Founded','Type of Ownership','Sector',
              'Revenue','Competitors','Easy Apply'],axis=1)

In [8]:
# Save cleaned data
df.to_csv(os.path.join(file_path,'datasciencejobs_cleaned.csv'),index=False)